In [1]:
import zstandard as zstd
import json
import pandas as pd
from pathlib import Path

In [2]:
def read_zst_to_dataframe(file_path, max_records=None):
    """
    Read a .zst compressed ndjson file and convert to pandas DataFrame
    
    Args:
        file_path: Path to the .zst file
        max_records: Optional limit on number of records to read (useful for testing)
    
    Returns:
        pandas DataFrame with the Reddit data
    """
    records = []
    
    with open(file_path, 'rb') as file_handle:
        reader = zstd.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
        buffer = ''
        
        while True:
            # Read chunk
            chunk = reader.read(2**20)  # 1MB chunks
            if not chunk:
                break
                
            # Decode and split into lines
            decoded_chunk = chunk.decode('utf-8')
            lines = (buffer + decoded_chunk).split('\n')
            
            # Process all complete lines (all except the last one)
            for line in lines[:-1]:
                if line.strip():  # Skip empty lines
                    try:
                        record = json.loads(line)
                        records.append(record)
                        
                        # Stop if we've hit the max_records limit
                        if max_records and len(records) >= max_records:
                            reader.close()
                            return pd.DataFrame(records)
                            
                    except json.JSONDecodeError:
                        # Skip malformed JSON lines
                        continue
            
            # Keep the incomplete line in buffer
            buffer = lines[-1]
            
        reader.close()
    
    return pd.DataFrame(records)

In [3]:
def read_all_zst_files(directory_path, file_pattern="RS_*.zst", max_records_per_file=None):
    """
    Read multiple .zst files and combine into one DataFrame
    
    Args:
        directory_path: Path to directory containing .zst files
        file_pattern: Pattern to match files (default: "RS_*.zst")
        max_records_per_file: Optional limit per file
    
    Returns:
        Combined pandas DataFrame
    """
    directory = Path(directory_path)
    zst_files = list(directory.glob(file_pattern))
    
    all_dataframes = []
    
    for file_path in sorted(zst_files):
        print(f"Reading {file_path.name}...")
        df = read_zst_to_dataframe(file_path, max_records_per_file)
        print(f"  Loaded {len(df)} records")
        all_dataframes.append(df)
    
    # Combine all dataframes
    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        print(f"\nTotal combined records: {len(combined_df)}")
        return combined_df
    else:
        print("No files found!")
        return pd.DataFrame()

In [4]:
# Example usage:

# 1. Read a single file (test with first 1000 records)
file_path = r"F:\reddit data\reddit\submissions\filtered data\wallstreetbets\RS_2021-06.zst"
df_sample = read_zst_to_dataframe(file_path, max_records=1000)

print("Sample data shape:", df_sample.shape)
print("\nColumns available:")
print(df_sample.columns.tolist())

print("\nFirst few records:")
print(df_sample.head())

Sample data shape: (1000, 89)

Columns available:
['all_awardings', 'allow_live_comments', 'archived', 'author', 'author_created_utc', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 'author_premium', 'can_gild', 'category', 'content_categories', 'contest_mode', 'created_utc', 'discussion_type', 'distinguished', 'domain', 'edited', 'gilded', 'gildings', 'hidden', 'hide_score', 'id', 'is_created_from_ads_ui', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media', 'media_embed', 'media_only', 'name', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'paren

In [5]:
# 2. Read all files (this might be a lot of data!)
directory_path = r"F:\reddit data\reddit\submissions\filtered data\wallstreetbets"
df_all = read_all_zst_files(directory_path)

print("Full dataset shape:", df_all.shape)
print("Memory usage:", df_all.memory_usage(deep=True).sum() / 1024**2, "MB")

Reading RS_2021-06.zst...
  Loaded 94987 records
Reading RS_2021-07.zst...
  Loaded 31560 records
Reading RS_2021-08.zst...
  Loaded 25759 records

Total combined records: 152306
Full dataset shape: (152306, 99)
Memory usage: 494.19725799560547 MB


In [ ]:
# # 3. For very large datasets, you might want to process in chunks
# def process_zst_in_chunks(file_path, chunk_size=10000):
#     """
#     Process large .zst files in chunks to avoid memory issues
#     """
#     chunk_count = 0
    
#     with open(file_path, 'rb') as file_handle:
#         reader = zstd.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
#         buffer = ''
#         records = []
        
#         while True:
#             chunk = reader.read(2**20)
#             if not chunk:
#                 break
                
#             decoded_chunk = chunk.decode('utf-8')
#             lines = (buffer + decoded_chunk).split('\n')
            
#             for line in lines[:-1]:
#                 if line.strip():
#                     try:
#                         record = json.loads(line)
#                         records.append(record)
                        
#                         if len(records) >= chunk_size:
#                             # Process this chunk
#                             df_chunk = pd.DataFrame(records)
#                             chunk_count += 1
#                             print(f"Processing chunk {chunk_count} with {len(df_chunk)} records")
                            
#                             # Do your analysis on df_chunk here
#                             # Example: print(df_chunk['title'].head())
                            
#                             records = []  # Reset for next chunk
                            
#                     except json.JSONDecodeError:
#                         continue
            
#             buffer = lines[-1]
        
#         # Process remaining records
#         if records:
#             df_chunk = pd.DataFrame(records)
#             chunk_count += 1
#             print(f"Processing final chunk {chunk_count} with {len(df_chunk)} records")
        
#         reader.close()

In [10]:
#load data with only useful fields
# def read_zst_filtered_fields(file_path, fields_to_keep, max_records=None):
#     records = []
    
#     with open(file_path, 'rb') as file_handle:
#         reader = zstd.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
#         buffer = ''
        
#         while True:
#             chunk = reader.read(2**20)
#             if not chunk:
#                 break
                
#             decoded_chunk = chunk.decode('utf-8')
#             lines = (buffer + decoded_chunk).split('\n')
            
#             for line in lines[:-1]:
#                 if line.strip():
#                     try:
#                         full_record = json.loads(line)
#                         # Only keep specified fields
#                         filtered_record = {field: full_record.get(field) for field in fields_to_keep if field in full_record}
#                         records.append(filtered_record)
                        
#                         if max_records and len(records) >= max_records:
#                             reader.close()
#                             return pd.DataFrame(records)
                            
#                     except json.JSONDecodeError:
#                         continue
            
#             buffer = lines[-1]
            
#         reader.close()
    
#     return pd.DataFrame(records)

def read_zst_filtered_fields(file_path, fields_to_keep, chunk_size=1024*1024):
    """
    Read a .zst file and return a DataFrame with only specified fields
    """
    records = []
    
    with open(file_path, 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        stream_reader = dctx.stream_reader(f)
        
        buffer = ""
        
        while True:
            chunk = stream_reader.read(chunk_size)
            if not chunk:
                break
            
            try:
                decoded_chunk = chunk.decode('utf-8')
            except UnicodeDecodeError:
                # Handle encoding errors gracefully
                decoded_chunk = chunk.decode('utf-8', errors='ignore')
                print(f"  Warning: Encountered encoding issues in {file_path.name}, some characters may be missing")
            
            lines = (buffer + decoded_chunk).split('\n')
            
            # Keep the last incomplete line in buffer
            buffer = lines[-1]
            
            for line in lines[:-1]:
                if line.strip():
                    try:
                        data = json.loads(line)
                        # Filter to keep only specified fields
                        filtered_data = {field: data.get(field) for field in fields_to_keep if field in data}
                        if filtered_data:  # Only add if we have some data
                            records.append(filtered_data)
                    except json.JSONDecodeError:
                        continue  # Skip malformed JSON lines
        
        # Process any remaining data in buffer
        if buffer.strip():
            try:
                data = json.loads(buffer)
                filtered_data = {field: data.get(field) for field in fields_to_keep if field in data}
                if filtered_data:
                    records.append(filtered_data)
            except json.JSONDecodeError:
                pass
    
    return pd.DataFrame(records)


In [11]:
# def read_all_files_filtered(directory_path, fields_to_keep, file_pattern="*.zst"):
#     """
#     Read all .zst files with only specified fields
#     """
#     from pathlib import Path
    
#     directory = Path(directory_path)
#     zst_files = list(directory.glob(file_pattern))
    
#     all_dataframes = []
    
#     for file_path in sorted(zst_files):
#         print(f"Reading {file_path.name} (filtered fields)...")
#         df = read_zst_filtered_fields(file_path, fields_to_keep)
#         print(f"  Loaded {len(df)} records with {len(df.columns)} columns")
#         all_dataframes.append(df)
    
#     if all_dataframes:
#         combined_df = pd.concat(all_dataframes, ignore_index=True)
#         print(f"\nTotal combined records: {len(combined_df)}")
#         print(f"Memory usage: {combined_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
#         return combined_df
#     else:
#         return pd.DataFrame()

def read_all_files_filtered(directory_path, fields_to_keep, file_pattern="*.zst"):
    """
    Read all .zst files with only specified fields
    """
    from pathlib import Path
    
    directory = Path(directory_path)
    zst_files = list(directory.glob(file_pattern))
    
    all_dataframes = []
    
    for file_path in sorted(zst_files):
        print(f"Reading {file_path.name} (filtered fields)...")
        try:
            df = read_zst_filtered_fields(file_path, fields_to_keep)
            print(f"  Loaded {len(df)} records with {len(df.columns)} columns")
            all_dataframes.append(df)
        except Exception as e:
            print(f"  Error reading {file_path.name}: {e}")
            print(f"  Skipping this file...")
            continue
    
    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        print(f"\nTotal combined records: {len(combined_df)}")
        print(f"Memory usage: {combined_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        return combined_df
    else:
        return pd.DataFrame()

In [12]:
# Example: Load all data with only essential fields
essential_fields = [
    'id', 'title', 'selftext', 'author', 'created_utc', 'score', 
    'num_comments', 'url', 'subreddit', 'upvote_ratio'
]


directory_path = r"F:\reddit data\reddit\submissions\filtered data\wallstreetbets"
df_wallstreetbets = read_all_files_filtered(directory_path, essential_fields)

Reading RC_2021-06.zst (filtered fields)...
  Loaded 2186516 records with 5 columns
Reading RC_2021-07.zst (filtered fields)...
  Loaded 1100198 records with 5 columns
Reading RC_2021-08.zst (filtered fields)...
  Loaded 1011270 records with 5 columns
Reading RS_2021-06.zst (filtered fields)...
  Loaded 94987 records with 10 columns
Reading RS_2021-07.zst (filtered fields)...
  Loaded 31560 records with 10 columns
Reading RS_2021-08.zst (filtered fields)...
  Loaded 25759 records with 10 columns

Total combined records: 4450290
Memory usage: 1460.1 MB


In [16]:
# testing data from one subreddit: r/wallstreetbets
# df_wallstreetbets
df_test = df_wallstreetbets[(df_wallstreetbets['selftext']!="[removed]") & (df_wallstreetbets['selftext']!="[deleted]") & (df_wallstreetbets['selftext'].isna()!=True)]
df_test.to_csv("test.csv",index=False)


In [17]:
df_test

,id,author,created_utc,score,subreddit,title,selftext,num_comments,url,upvote_ratio
4297989,npgy6l,TheMarkIII,1622505919,1,wallstreetbets,Caught a glimpse of AMC on its way to the moon!!,,1.0,https://i.redd.it/tuw6d5qlpj271.jpg,1.00
4297991,nph22g,macfern,1622506236,1,wallstreetbets,Wanda Group Fully Withdraws From AMC in Nearly...,,1.0,https://www.google.com/amp/s/www.theepochtimes...,1.00
4297993,nph34u,randyzmzzzz,1622506330,1,wallstreetbets,"AMC at 42nd street, NYC",,1.0,https://i.redd.it/9i2q5u6tqj271.jpg,1.00
4297996,nph718,TheHighness1,1622506662,1,wallstreetbets,Ape Strong together,,1.0,https://i.redd.it/fzn8646trj271.jpg,1.00
4297997,nph8m3,wowwkwkssiw,1622506792,1,wallstreetbets,Elon Musk Charged With Second Degree Murder,,1.0,https://theproffessionalbroker.blogspot.com/20...,1.00
...,...,...,...,...,...,...,...,...,...,...
4450281,pfhn3d,_disguy,1630452959,2117,wallstreetbets,GME has consistently spiked every T-17 days be...,,252.0,https://i.redd.it/zhjf7do34sk71.png,0.93
4450283,pfhpj2,Kobeblackmambattv,1630453193,75,wallstreetbets,$GENI DD | I'm either a $GENIus or a retard. 2...,&amp;#x200B;\n\nhttps://preview.redd.it/bfclvu...,45.0,https://www.reddit.com/r/wallstreetbets/commen...,0.89
4450284,pfhq3j,poohyawn,1630453246,2,wallstreetbets,Does anyone know what the first stock symbol i...,,14.0,https://i.redd.it/4jomom135sk71.jpg,0.61
4450285,pfhtyf,outphase84,1630453627,2,wallstreetbets,1.4K to 7.1K overnight on FIVN puts. Thanks ZM!,,2.0,https://i.redd.it/3drt4ft76sk71.jpg,1.00


In [ ]:
#storing data into disk
dfs.to_csv(r"F:\reddit data\reddit\submissions\filtered data\filteredsubandcoms.csv",index = False)